In [ ]:
include("../load.jl")

In [ ]:
include("../../data/speed_reducer.jl")

In [ ]:
gm = speed_reducer()
uniform_sample_and_eval!(gm)
set_param(gm, :ignore_accuracy, true)

In [ ]:
feasibility_sample(gm)

In [ ]:
learn_constraint!(gm)
add_tree_constraints!(gm)
optimize!(gm)
solution(gm)

In [ ]:
add_infeasibility_cuts!(gm)
optimize!(gm)
while abs(gm.cost[end] - gm.cost[end-1]) > 1e-10
    add_infeasibility_cuts!(gm)
    optimize!(gm)
end

In [ ]:
# Showing the iterations
show(stdout, MIME("text/latex"), gm.solution_history)

In [ ]:
# Checking that the solution is feasible
x = solution(gm)
for bbl in gm.bbls
    @assert bbl(x)[1] == bbl.feas_gap[end] >= 0
    println(bbl(x))
end

In [ ]:
m = speed_reducer()
using Ipopt
set_optimizer(m, Ipopt.Optimizer)
nonlinearize!(m)
optimize!(m)

In [ ]:
x = solution(m)

In [ ]:
for bbl in m.bbls
    @assert bbl(x)[1] == bbl.feas_gap[end] >= 0
    println(bbl(x))
end